In [18]:
import os
import random
import numpy as np
import pandas as pd
import collections
import math
import nltk

In [19]:
#label class 

#theta
#theta_training_in_loop
#theta_testing_in_loop
#avg_weight

################################
#a function here on old code line 38 

#for testing part- line 22. turn that into an argmax to generate 
# predicted label 

In [20]:
class LabelClass:
    
    #the constructor function, the things here WILL need to be give 
    #to even initialize the label class 
    def __init__(self, label, number_of_zeros_needed):
        self.label = label 
        self.number_of_zeros_needed= number_of_zeros_needed

        #initializing the things that need to happen 
        self.theta = np.zeros(number_of_zeros_needed)
        self.running_total = np.zeros(number_of_zeros_needed)

    #calc score for each label as an intermidiate value 
    def calc_score(self, x_vector):
        return np.dot(x_vector, self.theta) 
    
    #this is new- this is what we are using within loop throughout the loops 
    def update_running_total(self):
        self.running_total+=self.theta
    
    #this is just a calculation 
    #once at the very end 
    def calc_avg_weight(self, count):
        self.avg_weight= self.running_total/count


    def calc_test_score(self,x_vector):
        return np.dot(x_vector, self.avg_weight) 


    #this is gonna give you a true or false 

    #the label input here is the predicted label 
    def is_label(self, label):
        return self.label == label

In [21]:
uri_train  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'

df_train = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

labels= ['true', "false", "half-true", "barely-true"]
other_df = pd.DataFrame({"label": labels})

df_linear = df_train[df_train.label.isin(other_df.label)]

df_linear_1= df_linear.reset_index()
df_linear_1['statement'] = df_linear_1['statement'].str.lower()
#df_linear_1['statement']= df_linear_1['statement'].str.split()


In [22]:
uri_test  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'

df_test = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

other_df_1 = pd.DataFrame({"label": labels})

df_linear_test = df_test[df_test.label.isin(other_df_1.label)]

df_linear_test_1= df_linear_test.reset_index()
df_linear_test_1['statement'] = df_linear_test_1['statement'].str.lower()

In [23]:
df_linear_1['tokenized_statement'] = df_linear_1.apply(lambda row: nltk.word_tokenize(row['statement']), axis=1)
df_linear_test_1['tokenized_statement'] = df_linear_test_1.apply(lambda row: nltk.word_tokenize(row['statement']), axis=1)


In [24]:
#build a vocab 

#all the unique words in a dataset 
word_set = set()

for idx, row in df_linear_1.iterrows():
    word_set.update(row['tokenized_statement'])


In [25]:
#this is a look up table 
vocab = {word:idx for idx,word in enumerate(word_set)}

In [26]:
number_of_zeros_needed = len(vocab)

label_dict= {}

for i in labels: 

    label_dict[i]= LabelClass(i,number_of_zeros_needed)

In [44]:
print(label_dict)

{'true': <__main__.LabelClass object at 0x7f18782113a0>, 'false': <__main__.LabelClass object at 0x7f1878211280>, 'half-true': <__main__.LabelClass object at 0x7f18782b5be0>, 'barely-true': <__main__.LabelClass object at 0x7f187824beb0>}


In [27]:
def vector_generator(sequence_toekns, vocab): 

    return_vector= np.zeros(number_of_zeros_needed)

    #set some positions within that vector 
    for token in  sequence_toekns: 
        pos = vocab.get(token)
        if pos is not None:
            return_vector[pos] +=1



    return(return_vector)


In [60]:
def avg_perceptron(df_train, df_test):

    count = 0

    while count<180000:

        df_train_sample= df_train.sample()

        #get us the lable 
        actual_label = df_train_sample["label"].iat[0]

        #one hot vector for sentence 
        x_vector= vector_generator(df_train_sample["tokenized_statement"].iat[0],vocab)

        #To James- why the .dot operation?? 

        #for i in range(len(theta)):

        #calc scores for algo 

        label_scores = []

        #this is new 
        for label in labels:
            #label_obj is the obj from that dict 
            label_obj = label_dict[label]

            #    def calc_score(self, x_vector):
            #        return np.dot(x_vector, self.theta) 

            label_scores.append(label_obj.calc_score(x_vector))

        #this really is just getting the index for the highest score 
        max_score_index = np.argmax(label_scores)

        #to figure out which label it really is 
        predicted_label = labels[max_score_index]


        # predicted_label = "true" if theta_true_in_loop >= theta_false_in_loop else "false"

        if actual_label != predicted_label:


            #this is the same thing 
            for label_obj in label_dict.values():
                #this is where that TRUE/FALSE return function within the Labelclass comes in 
                if label_obj.is_label(actual_label):
                    label_obj.theta += x_vector
                else:
                    label_obj.theta -= x_vector


        #add theta to your total obj
        for label_obj in label_dict.values():
            label_obj.update_running_total()
        count+=1 

    #this is a python syntax 
    for label_obj in label_dict.values():
        label_obj.calc_avg_weight(count)



In [61]:
avg_perceptron(df_linear_1, df_linear_test_1)

In [62]:
df_test = df_linear_test_1

to_calc =[]


for i,row in df_test.iterrows():

    #get us the lable 
    actual_label = row["label"]

    #one hot vector for sentence 
    x_vector= vector_generator(row["tokenized_statement"],vocab)

################################################################################3
    #for i in range(len(theta)):

    #calc scores for algo 
    label_scores = []

    #this is new 
    for label in labels:
        
        label_obj = label_dict[label]

        label_scores.append(label_obj.calc_test_score(x_vector))

    max_score_index = np.argmax(label_scores)
    predicted_label = labels[max_score_index]


########################################################################################3
    #this is to calc accurracy later 
    if predicted_label == row["label"]: 
        to_calc.append(1)
    else: 
        to_calc.append(0)

    ###########################################################################

count_correct = to_calc.count(1)
rows_count = df_linear_test_1.shape[0]
accuracy = count_correct/rows_count

print(accuracy)



0.7788681274364834
